## Imports 

In [1]:
from bs4 import BeautifulSoup # for webscraping 
import requests # for making http requests

from urllib import urlopen
import pymysql.cursors



In [3]:
result = requests.get("https://fr.wikipedia.org/wiki/R%C3%A9gions_du_Maroc")

In [4]:
c = result.content # get the content of the page 

In [5]:
soup = BeautifulSoup(c) # new instance of beautiful soup with the content of the page 

/home/neo/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/neo/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
data = [] 

In [8]:
table = soup.find_all('table', { 'class': 'wikitable' })[1] # find all table tags in the page 

In [9]:
""" This part is just a demo of how to  FIX the unicode problem, try to print e.text to see how it looks like  """ 
headers = table.find_all('th')
th = []
import unicodedata
th = map(lambda e: unicodedata.normalize('NFKD', e.text).encode('ascii', 'ignore'), headers)

In [10]:
tds = [] 
regions = []
tds = table.find_all('td')
for td in tds  : 
    if td.has_attr("rowspan"): 
        regions.append(unicodedata.normalize('NFKD',td.find('a').text).encode("ascii", "ignore"))
        

In [11]:
provinces = {}

In [12]:
regions 

['Tanger-Tetouan-Al Hoceima',
 "L'Oriental",
 'Fes-Meknes',
 'Rabat-Sale-Kenitra',
 'Beni Mellal-Khenifra',
 'Casablanca-Settat',
 'Marrakech-Safi',
 'Draa-Tafilalet',
 'Souss-Massa',
 'Guelmim-Oued Noun',
 'Laayoune-Sakia El Hamra',
 'Dakhla-Oued Ed-Dahab']

In [11]:
"""
This is a bit tricky, I need to delete the first td, because if not, i would have an extra element in my provinces dict, 
with the key 1 and no value, because of the first condition ' td.has_attr('rowspan') ' below.
There is always a better solution :)  .
"""
del tds[0]

In [12]:
""" 
 Here we're grouping all the provinces by the id of the region, which is in this case is the counter variable.
"""
counter = 1
t = []
for td in tds: 
    if ( td.has_attr('rowspan')  ): 
        provinces[counter] = t 
        counter += 1 
        t = []
    else : 
        t.append(unicodedata.normalize('NFKD', td.find('a').text).encode("ascii", "ignore"))
    provinces[counter] = t 
""" 
The keys of provinces dict, are in this case the foreing key in the relationship with regions.
"""

' \nThe keys of provinces dict, are in this case the foreing key in the relationship with regions.\n'

In [16]:
f = open("regions", "a")

In [17]:

for region_index, region_name in enumerate(regions): 
    f.write("{} \n".format(region_name)) 
f.close() 

In [21]:
value = 3
for region_index, region_name  in enumerate(regions): 
    f.write("<optgroup label=\"{}\"> \n".format(region_name))
    value += 1
    for province in provinces[region_index+1]:
        f.write("\t <option value=\"{}\">{}</option> \n".format(value, province)) 
        value += 1 
    f.write("</optgroup> \n")
f.close()

In [22]:
""" 
Establish the connection with your database.
You need to change the host, user, password, and the db variables in regard of your environnement.
"""
connection = pymysql.connect(host='localhost',
                             user='ayoubensalem',
                             password='19641995',
                             db='tamkinedb',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [21]:
try:
    with connection.cursor() as cursor:
        for region in regions:
            sql = "INSERT INTO `regions` (`region`) VALUES (%s)"
            cursor.execute(sql, (region))
    connection.commit()
finally:
    connection.close()

In [23]:
""" 
region is a foreign key that references Regions(id)
"""
try:
    with connection.cursor() as cursor:
        for k in provinces:
            for p in provinces[k]: 
                sql = "INSERT INTO `provinces` (`province`,`region_id`) VALUES (%s, %s)"
                cursor.execute(sql, (p,k))
    connection.commit()
finally:
    connection.close()